## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [1]:
##### PRODUCTOS #####
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self.codigo = codigo
        self.nombre = nombre
        self.tipo = tipo
        self.precio = precio
        self.cantidad = cantidad

    def obtenerPrecio(self):
        return self._precio

    def ponerPrecio(self, nuevoPrecio):
        match nuevoPrecio:
            case nuevoPrecio if 10 <= nuevoPrecio <= 10000:
                self._precio = nuevoPrecio
            case _:
                pass 
    def obtenerCantidad(self):
        return self._cantidad

    def ponerCantidad(self, nuevaCantidad):
        match nuevaCantidad:
            case nuevaCantidad if 0 <= nuevaCantidad <= 1000:
                self._cantidad = nuevaCantidad
            case _:
                pass 

    def valorTotal(self):
        return self._precio * self._cantidad

    precio = property(obtenerPrecio, ponerPrecio)
    cantidad = property(obtenerCantidad, ponerCantidad)

In [2]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [3]:
##### OFERTA #####
class Oferta:
    def __init__(self, des=[], codigos=[], tipos=[]):
        self.descripcion = des[:100]
        self.codigos = codigos
        self.tipos = tipos

    def esAplicable(self, producto, cantidad):
        return producto.codigo in self.codigos or producto.tipo in self.tipos

    def aplicar(self, producto, cantidad):
        pass


##### DESCUENTO #####
class OfertaDescuento(Oferta):
    def __init__(self, descuento, codigos=[], tipos=[]):
        super().__init__(f"Descuento {descuento}%", codigos, tipos)
        self.descuento = descuento

    def esAplicable(self, producto, cantidad):
        if self.codigos and producto.codigo in self.codigos:
            return True
        if self.tipos and producto.tipo in self.tipos:
            return True
        return False

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            descuento = (self.descuento / 100) * producto.precio * cantidad
            return descuento
        return 0


##### OFERTA 2X1 #####
class Oferta2x1(Oferta):
    def __init__(self, des="Sin descripción", codigos=[], tipos=[]):
        super().__init__(des, codigos, tipos)

    def esAplicable(self, producto, cantidad):
        if self.codigos:
            return producto.codigo in self.codigos and cantidad >= 2
        return super().esAplicable(producto, cantidad) and cantidad >= 2
    

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            pares = cantidad // 2
            descuento = pares * producto.precio
            return descuento
        return 0  

In [4]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0 

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

assert o2x1.esAplicable(p2, 0) == False 
assert o2x1.esAplicable(p2, 1) == False 
assert o2x1.esAplicable(p2, 2) == True 
assert o2x1.esAplicable(p2, 3) == True 

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [5]:
import csv

##### CATALOGO #####
class Catalogo:
    def __init__(self):
        self.dicProductos = {}
        self.ofertas = []

    def agregar(self, *productos):
        for producto in productos:
            self.dicProductos[producto.codigo] = producto

    def buscar(self, codigo):
        return self.dicProductos.get(codigo, None)

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def buscarOferta(self, producto, cantidad):
        for i in self.ofertas:
            if i.esAplicable(producto, cantidad):
                return i
        return None

    def calcularDescuento(self, producto, cantidad):
        for oferta in self.ofertas:
            if oferta.esAplicable(producto, cantidad):
                descuento = oferta.calcularDescuento(producto, cantidad)
                if descuento > 0:
                    return descuento
        return 0


    def vender(self, producto, cantidad):
        if producto.cantidad >= cantidad:
            producto.cantidad -= cantidad

    def obtenerCantidadProductos(self):
        return len(self.dicProductos)

    def obtenerCantidadUnidades(self):
        return sum(p.cantidad for p in self.dicProductos.values())

    def obtenerValorTotal(self):
        return sum(p.valorTotal() for p in self.dicProductos.values())

    def __getattr__(self, nmAtributo):
        if nmAtributo == "cantidadProductos":
            return self.obtenerCantidadProductos()
        elif nmAtributo == "cantidadUnidades":
            return self.obtenerCantidadUnidades()
        elif nmAtributo == "valorTotal":
            return self.obtenerValorTotal()

    def informe(self):
        dicTipos = {}
        for p in self.dicProductos.values():
            if p.tipo not in dicTipos:
                dicTipos[p.tipo] = {"unidades": 0, "valor": 0}
            dicTipos[p.tipo]["unidades"] += p.cantidad
            dicTipos[p.tipo]["valor"] += p.valorTotal()

        informe = f"INFORME CATALOGO\nCantidad de productos: {self.obtenerCantidadProductos()}\n"
        informe += f"Cantidad de unidades: {self.obtenerCantidadUnidades()}\n"
        informe += f"Valor total: ${self.obtenerValorTotal():.2f}\n"
        informe += f"Precio Promedio: \n"
        
        for t, d in dicTipos.items():
            promedio = d["valor"] / d["unidades"] if d["unidades"] > 0 else 0
            informe += f"  - {t}: ${promedio:.2f}\n"

        informe += "Tipos de productos: \n"
        for t, d in dicTipos.items():
            informe += f"  - {t}: {d['unidades']}u x ${d['valor']:.2f}\n"

        informe += "Ofertas: \n"
        for o in self.ofertas:
            informe += f"  - {o.descripcion}\n"
    
        informe += "Oferta 2x1: \n"
        for o in self.ofertas:
            informe += f"  - {o.descripcion}\n"
        return informe

    def guardar(self, nmArchivo):
        with open(nmArchivo, "w") as file:
            wrt = csv.writer(file)
            wrt.writerow(["codigo", "nombre", "precio", "tipo", "cantidad"])
            for i in self.dicProductos.values():
                wrt.writerow([i.codigo, i.nombre, i.precio, i.tipo, i.cantidad])

    def leer(nmArchivo):
        catalogo = Catalogo()  
        with open(nmArchivo, "r") as file:
            rdr = csv.DictReader(file)
            for i in rdr:
                rdProducto = Producto(
                    codigo=i["codigo"],
                    nombre=i["nombre"],
                    precio=int(i["precio"]),
                    tipo=i["tipo"],
                    cantidad=int(i["cantidad"])
                )
                catalogo.agregar(rdProducto)  
        return catalogo  



In [6]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0

catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [7]:
##### CLIENTE #####
class Cliente:
    def __init__(self, nombre, cuit):
        self._nombre = nombre
        self._cuit = cuit

    def __getattr__(self, nm):
        if nm == "nombre":
            return self._nombre
        elif nm == "cuit":
            return self._cuit

    def __setattr__(self, nm, valor):
        if nm == "nombre":
            match len(valor):
                case longitud if 1 <= longitud <= 100:
                    self.__dict__["_nombre"] = valor
                case _:
                    pass
        elif nm == "cuit":
            match self.validarCuit(valor):
                case True:
                    self.__dict__["_cuit"] = valor
                case False:
                    pass
        else:
            super().__setattr__(nm, valor)


    def validarCuit(self, cuit):
        p = cuit.split("-")
        match p:
            case [p1, p2, p3] if p1.isdigit() and p2.isdigit() and p3.isdigit():
                if len(p1) == 2 and len(p2) == 8 and len(p3) == 1:
                    return True
            case _:
                return False

In [8]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [9]:
##### FACTURA #####
class Factura:
    numSecuencial = 0

    def __init__(self, catalogo, cliente):
        self.numero = Factura.nuevoNumero()
        self.fecha = "01/10/2024"
        self.cliente = cliente
        self.catalogo = catalogo
        self.productos = []
        self.subtotal = 0
        self.descuentos = 0

    def nuevoNumero():
        Factura.numSecuencial += 1
        return Factura.numSecuencial

    def ultimaFactura(numero):
        Factura.numSecuencial = numero

    
    def agregar(self, producto, cantidad):
        for i, (p, c, st, d) in enumerate(self.productos):
            if p.codigo == producto.codigo:
                nuevaCantidad = c + cantidad
                nuevoDescuento = self.catalogo.calcularDescuento(producto, nuevaCantidad)
                nuevoSubtotal = producto.precio * nuevaCantidad
                self.subtotal -= st
                self.descuentos -= d
                self.productos[i] = (producto, nuevaCantidad, nuevoSubtotal, nuevoDescuento)
                self.subtotal += nuevoSubtotal
                self.descuentos += nuevoDescuento
                producto.cantidad -= cantidad
                return
            
        descuento = self.catalogo.calcularDescuento(producto, cantidad)
        sbtlProducto = producto.precio * cantidad
        self.productos.append((producto, cantidad, sbtlProducto, descuento))
        self.subtotal += sbtlProducto
        self.descuentos += descuento
        producto.cantidad -= cantidad

    def obtenerCantidadProductos(self):
        return len(self.productos)

    def obtenerCantidadUnidades(self):
        return sum(cantidad for _, cantidad, _, _ in self.productos)

    def obtenerTotal(self):
        return self.subtotal - self.descuentos


    def __getattr__(self, nm):
        match nm:
            case "cantidadProductos":
                return self.obtenerCantidadProductos()
            case "cantidadUnidades":
                return self.obtenerCantidadUnidades()
            case "total":
                return self.obtenerTotal()
            case _:
                pass


    def imprimir(self):
        factura = f"Factura: {self.numero}\nFecha  : {self.fecha}\nCliente: {self.cliente.nombre} ({self.cliente.cuit})\n"
        for p, c, st, d in self.productos:
            factura += f"- {c} {p.nombre} x ${p.precio} = ${st:.2f}\n"
            if d > 0:
                oferta_aplicada = self.catalogo.buscarOferta(p, c)
                if oferta_aplicada:
                    if isinstance(oferta_aplicada, Oferta2x1):
                        factura += f"      Oferta 2x1: -${d:.2f}\n"
                    else:
                        factura += f"      {oferta_aplicada.descripcion}: -${d:.2f}\n"
        factura += f"\nSubtotal: ${self.subtotal:.2f}\nDescuento 10%: ${self.descuentos:.2f}\nTOTAL: ${self.obtenerTotal():.2f}\n"
        return factura


In [10]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [11]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
